## 探索
加载数据

In [ ]:
import para
from utilities import DataLoader
dl = DataLoader()
l = []
for machine_num in range(33):
    raw_data = dl(para.train_data,machine_num) #加载数据
    l.append(len(raw_data))
    print(str(machine_num+1)+'Done.')
    
# print(raw_data.shape)
# print(miss[0].shape)
# print(miss[1].shape)

In [ ]:
print(l)

In [ ]:
index = 346449
machine_num = 0
for i in range(len(l)):
    if index>=l[i]:
        index-=l[i]
    else:
        machine_num = i
        break
print(machine_num,index)
print(machine_num != None)        

In [ ]:
import para
import csv
import numpy as np
import matplotlib.pyplot as plt

# print(t)
for i in range(para.machine_num):
    file_path = para.train_data +  '{0:03d}'.format(i+1) + '/201807.csv'
    with open(file_path,'r') as file:
        data = csv.reader(file)
        data.__next__()
        full_data = []
        p_miss_data = []
        for record in data:
            if any([r==''for r in record]):
                p_miss_data.append(record)
            else:
                full_data.append(record)
        raw_data_str = full_data
    
    break
    
# record to be filled
file_path = para.train_data+'template_submit_result.csv'
all_miss_data = []
with open(file_path,'r') as file:
    data = csv.reader(file)
    data.__next__()
    for record in data:
        if record[1] == '1':
            all_miss_data.append(record)

无缺失字段记录

In [ ]:
%matplotlib inline
t = ['M'] + ['i4'] + ['f4']*15+['i4']+['f4']*3+['i4']+['f4']*26+['i4']+['f4']*5+['i4']+['f4']*12+['i4']+['f4']*2

raw_data_str = np.array(raw_data_str)
raw_data = []

for i in range(70):
    s = raw_data_str[:,i].astype(t[i])
    if i != 0 and i !=1:
        plt.figure()
        plt.hist(s)
        plt.xlabel('var num {0}, type {5}, min {1:.2f}, max {2:.2f}, mean {3:.2f}, std {4:.2f}'.format(i-1,np.min(s),np.max(s),np.mean(s),np.std(s),t[i]))
        plt.show()
    raw_data.append(s)
#     print(s[0],s.dtype)

# print(raw_data[0],raw_data.dtype)

时序性测试

In [ ]:
# plt.figure(figsize=(16,8))
for i in range(2,70):
    plt.plot(raw_data[i],label=str(i-1))
    plt.xlabel('var'+str(i-1))
    plt.show()

有缺失字段记录

In [ ]:
p_miss_count = [0 for _ in range(70)]
p_miss_total = len(p_miss_data)
# print(p_miss_total)
for r in p_miss_data:
    for i in range(len(r)):
        if r[i]=='':
            p_miss_count[i]+=1
p_miss_count = np.array(p_miss_count)
plt.figure(figsize=(16,8))
plt.bar(np.array(range(70)),p_miss_count/p_miss_total)
plt.show()

f_miss_t = []    
p_miss_t = []
for record in all_miss_data:
    flag = False
    for p_r in p_miss_data:
        if record[0]==p_r[0]:
            flag = True
            break
    if flag:
        p_miss_t.append(record[0])
    else:
        f_miss_t.append(record[0])    
plt.figure(figsize=(16,8))
plt.plot(raw_data[0],[1 for _ in range(len(raw_data[0]))],'b.',label='full')
plt.plot(np.array(p_miss_t,dtype='M'),[0.9 for _ in range(len(p_miss_t))],'r.',label='p_miss')
plt.plot(np.array(f_miss_t,dtype='M'),[0.8 for _ in range(len(f_miss_t))],'g.',label='f_miss')
plt.legend(loc='best')
plt.show()
    

预处理
1. 对float类型做standardization (x-mean)/std 
2. 对int，book类型做one_hot

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
# class used for pre-process data 
class preproc:
    def __init__(self, data, types):
        self.features = [True if t=='f4' else False for t in types] # 68个字段预处理分类操作
        self.continuous, self.discrete = self.classify(data)
        self.proc_cont = self.standardization(self.continuous)
        self.proc_disc = self.one_hot(self.discrete)
        
    def classify(self, data):
        return data[:,self.features], data[:,np.logical_not(self.features)]
        
    def standardization(self, data):
        self.stda = StandardScaler()
        return self.stda.fit_transform(data)
        
    def one_hot(self,data):
        self.enc = OneHotEncoder()
        return self.enc.fit_transform(data)
        

In [ ]:
#test
data = np.array(raw_data).T[:,2:]
pp = preproc(data,t[2:])
inputs = np.concatenate((pp.proc_cont,pp.proc_disc.todense()),axis=1)
print(inputs.shape)


## 实验

In [ ]:
%run exp.py